In [6]:
# 단층 신경망 구성
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_resource_variables()

import numpy as np

# 학습용 데이터 구성, [털, 날개]
x_data = np.array([[0,0],
                  [1,0],
                  [1,1],
                  [0,0],
                  [0,0],
                  [0,1]])
# 레이블 데이터 구성
# 원핫인코딩으로 작성
# 기타=[1,0,0], 포유류=[0,1,0], 조류=[0,0,1]
y_data = np.array([
    [1,0,0], #기타
    [0,1,0], #포유류
    [0,0,1], #조류
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

# 신경망 모델 구성
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)
# 입출력 결과의 차원수만 잘 나타내주면 되는듯 하다.
W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

L=tf.add(tf.matmul(X,W),b)
L=tf.nn.relu(L)

# 결과값의 전체합이 1이 되도록 만들어줌
model = tf.nn.softmax(L)

# 원핫인코딩->교차엔트로피 사용
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

## 학습
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행
for step in range(300):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습도중 10번에 1번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
# 예측값인 model을 바로 출력하면 [0.2 0.7 0.1]과 같은 확률로 나와
# 요소중 가장 큰값의 인덱스를 찾아주는 argmax 사용
# argmax : 결과중 가장 큰 인덱스를 찾아줌
# 예시 : [[0 1 0] [1 0 0]]-> [1 0]
#        [[0.2 0.7 0.1] [0.9 0.1 0.]]-> [1 0]
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 정확도 출력
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: {}'.format(sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data})))

10 1.1173695
20 1.1131747
30 1.1092042
40 1.1052849
50 1.1014982
60 1.097874
70 1.0964313
80 1.0951158
90 1.0938008
100 1.092506
110 1.0912308
120 1.0899748
130 1.0887376
140 1.0875187
150 1.0862944
160 1.0850811
170 1.0838892
180 1.0827408
190 1.0816087
200 1.0804925
210 1.0793718
220 1.078231
230 1.0771617
240 1.0761073
250 1.0750216
260 1.0739671
270 1.0729555
280 1.0722547
290 1.0717468
300 1.0712605
예측값: [0 1 1 0 0 0]
실제값: [0 1 2 0 0 2]
정확도: 66.66667175292969


In [7]:
# 심층 신경망 구성
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_resource_variables()

import numpy as np

# 학습용 데이터 구성, [털, 날개]
x_data = np.array([[0,0],
                  [1,0],
                  [1,1],
                  [0,0],
                  [0,0],
                  [0,1]])
# 레이블 데이터 구성
# 원핫인코딩으로 작성
# 기타=[1,0,0], 포유류=[0,1,0], 조류=[0,0,1]
y_data = np.array([
    [1,0,0], #기타
    [0,1,0], #포유류
    [0,0,1], #조류
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

# 신경망 모델 구성
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

# w1=[2 10] -> [피쳐, 은닉층 뉴런수]
# w2=[10 3] -> [은닉층 뉴런 수, 분류 수]
W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 첫번째 가중치, 편향, 활성화 함수 적용
L1=tf.add(tf.matmul(X,W1),b1)
L1=tf.nn.relu(L1)
# 출럭층을 만들기 위해 두번째 가중치와 편향을 적용
# 출력층에서 활성화 함수 사용 x
model = tf.add(tf.matmul(L1, W2), b2)

# tensorflow가 기본 제공하는 교차 엔트로피 함수 이용
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

## 학습
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습도중 10번에 1번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
# 예측값인 model을 바로 출력하면 [0.2 0.7 0.1]과 같은 확률로 나와
# 요소중 가장 큰값의 인덱스를 찾아주는 argmax 사용
# argmax : 결과중 가장 큰 인덱스를 찾아줌
# 예시 : [[0 1 0] [1 0 0]]-> [1 0]
#        [[0.2 0.7 0.1] [0.9 0.1 0.]]-> [1 0]
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 정확도 출력
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: {}'.format(sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data})))

10 0.8981491
20 0.721123
30 0.5974434
40 0.49540982
50 0.41119847
60 0.34063277
70 0.28362662
80 0.23856997
90 0.20303933
100 0.17486246
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.0
